In [1]:
import numpy as np
from darli.model import Functional
from darli.robots import quadruped
from darli.backend import CasadiBackend, JointType
from darli.backend.liecasadi.so3 import SO3

import casadi as cs

In [2]:
%%capture
!pip3 install robot_descriptions
# from robot_descriptions import go2_description, go2_mj_description
from robot_descriptions import a1_description, a1_mj_description

In [16]:
# from mujoco_simulator.model_builder import ModelBuilder
# from mujoco_simulator.simulator import MujocoRender
# model_wrapper = ModelBuilder(a1_mj_description.MJCF_PATH)
# model_wrapper.add_arena()
# model_wrapper.build()
# ren = MujocoRender(model=model_wrapper, 
#                    start=True, 
#                    convention='pin')
import mujoco

model = mujoco.MjModel.from_xml_path(a1_mj_description.MJCF_PATH)
renderer = mujoco.Renderer(model)#, width=800, height=600)

In [17]:
model = quadruped(Functional, CasadiBackend,
                  a1_mj_description.URDF_PATH,
                  foots={'fl': "FL_foot", 
                         'fr': "FR_foot", 
                         'rl': "RL_foot", 
                         'rr': "RR_foot"}, 
                  root_joint=JointType.FREE_FLYER)

In [18]:
model.nq

30

In [19]:
model.bodies.keys()

dict_keys(['fl', 'fr', 'rl', 'rr'])

Define the variables in tangent space and get the configuration via exponential:

In [20]:
opti_problem = cs.Opti()
dq = opti_problem.variable(model.nv)

In [21]:
q0 = opti_problem.parameter(model.nq)

In [22]:
q = model.backend.integrate_configuration(q0, dq)

TypeError: unsupported operand type(s) for +: 'MX' and 'MX'

In [11]:
for body in model.bodies.values():
    name = body.name
    if name != 'torso':
        initial_foot_pos = model.body(name).position(q0)
        initial_foot_pos[2] = 0 
        opti_problem.subject_to(model.body(name).position(q) == initial_foot_pos)

NameError: name 'q' is not defined

In [11]:
torso_pos = q[:3]
torso_xyzw = q[3:7]

In [12]:
desired_pos = opti_problem.parameter(3)
desired_xyzw = opti_problem.parameter(4)

In [13]:
so3_error = SO3(xyzw=torso_xyzw).distance(SO3(xyzw = desired_xyzw))
pos_cost = cs.sumsqr(torso_pos - desired_pos)
so3_cost = 0.1*so3_error

In [14]:
cost = pos_cost + so3_cost #+ gravity_cost
opti_problem.minimize(cost)
opti_problem.subject_to(opti_problem.bounded(model.q_min, q[7:], model.q_max))
opti_problem.solver('ipopt')

In [15]:
pos_des = [0.0, 0.0, 0.3]
xyzw_des = SO3.qy(-0.3).xyzw

In [16]:
q_init = np.zeros(model.nq)
q_init[6] = 1
opti_problem.set_initial(dq, np.zeros(model.nv))
opti_problem.set_value(q0,q_init)
opti_problem.set_value(desired_pos,pos_des)
opti_problem.set_value(desired_xyzw, xyzw_des)

In [17]:
opti_problem.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       84
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       69

Total number of variables............................:       18
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality c

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 18)
  #parameters: 3 (np = 26)
  #constraints: 5 (ng = 24)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [18]:
dq_opt = opti_problem.value(dq)
dq_opt

array([ 1.50861122e-09, -1.98726106e-17,  2.99999981e-01,  3.50458348e-17,
       -2.99999964e-01, -4.68872098e-17,  6.85679391e-17,  8.51845284e-01,
       -1.15205211e+00, -4.26842390e-17,  8.51845284e-01, -1.15205211e+00,
        2.21586461e-17,  1.29933072e+00, -1.92755251e+00, -3.96244075e-17,
        1.29933072e+00, -1.92755251e+00])

In [19]:
q_opt = opti_problem.value(q)
q_opt

array([ 1.50861122e-09, -1.98726106e-17,  2.99999981e-01,  1.74572804e-17,
       -1.49438114e-01, -2.33557903e-17,  9.88771081e-01,  6.85679391e-17,
        8.51845284e-01, -1.15205211e+00, -4.26842390e-17,  8.51845284e-01,
       -1.15205211e+00,  2.21586461e-17,  1.29933072e+00, -1.92755251e+00,
       -3.96244075e-17,  1.29933072e+00, -1.92755251e+00])

In [20]:
q_opt[:3]

array([ 1.50861122e-09, -1.98726106e-17,  2.99999981e-01])

In [21]:
ren.set_state(q_opt)

In [22]:
q_opt[3:7]

array([ 1.74572804e-17, -1.49438114e-01, -2.33557903e-17,  9.88771081e-01])

In [23]:
xyzw_des

DM([0, -0.149438, 0, 0.988771])

In [24]:
pos_des = [0.0, 0.0, 0.28]
xyzw_des = [0.0, 0.0, 0.0, 1.0]

In [25]:
opti_problem.set_initial(dq, dq_opt)
opti_problem.set_value(q0,q_opt)
opti_problem.set_value(desired_pos, pos_des)
opti_problem.set_value(desired_xyzw, xyzw_des)

In [26]:
opti_problem.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       84
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       69

Total number of variables............................:       18
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       12
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.3839997e-01 1.13e+00 3.05e-02  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 18)
  #parameters: 3 (np = 26)
  #constraints: 5 (ng = 24)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [27]:
q_opt = opti_problem.value(q)
q_opt

array([ 4.69128390e-09,  2.73965394e-21,  2.79999987e-01, -3.32728263e-22,
       -1.10761687e-08, -2.19426828e-21,  1.00000000e+00, -6.10290905e-17,
        8.53596443e-01, -1.70719281e+00, -8.71244726e-20,  8.53596443e-01,
       -1.70719281e+00, -2.07673737e-17,  8.53596470e-01, -1.70719286e+00,
        1.50630054e-19,  8.53596470e-01, -1.70719286e+00])

In [28]:
q_opt[:3]

array([4.69128390e-09, 2.73965394e-21, 2.79999987e-01])

In [29]:
q_opt[3:7]

array([-3.32728263e-22, -1.10761687e-08, -2.19426828e-21,  1.00000000e+00])

In [30]:

cost = pos_cost + 0.5*so3_cost + 0.0001*cs.sumsqr(dq)
opti_problem.minimize(cost)
opti_problem.solver(
            "ipopt",
            {
                "ipopt.print_level": 0,
                # 'ipopt.max_iter': 3,
                "print_time": 0,
                "ipopt.warm_start_init_point": "yes"
            },
        )


In [31]:
# opti_problem.solver('sqpmethod')
from time import perf_counter, sleep

xyzw_des = [0,0,0,1]
for j in range(2):
    for i in range(600):
        t1 = perf_counter()
        coordinate = 0.08*np.sin(2*np.pi*i/100) 
        if i >=0:
            pos_des = [coordinate, 0.0, 0.3]
        if i >=100:
            pos_des = [0, coordinate, 0.3]
        if i >=200:
            pos_des = [0, 0.0, 0.3 + coordinate]
        if i >=300:
            pos_des = [0, 0.0, 0.3]
            xyzw_des = SO3.qx(6*coordinate).xyzw
        if i >=400:
            xyzw_des = SO3.qy(6*coordinate).xyzw
        if i >=500:
            xyzw_des = SO3.qz(6*coordinate).xyzw

        
        opti_problem.set_value(desired_pos,pos_des)
        opti_problem.set_value(desired_xyzw,xyzw_des)
        opti_problem.set_initial(dq,dq_opt)
        try:
            sol = opti_problem.solve()
            dq_opt = opti_problem.value(dq)
            q_opt = opti_problem.value(q)
        except Exception as e:
            dq_opt = opti_problem.debug.value(dq)
            q_opt = opti_problem.debug.value(q)
        
        dq_opt = opti_problem.value(dq)
        q_opt = opti_problem.value(q)
        # print(q_opt)
        
        t2 = perf_counter()
        print(1000*(t2 - t1))
        ren.set_state(q_opt)

        ren.markers[0](position=q_opt[:3].copy(),
                            color=[1, 0, 0, 0.5],
                            size=0.02)
        
        ren.markers[1](position=np.array(pos_des).copy(),
                            color=[0, 0, 1, 0.5],
                            size=0.022)
        sleep(0.01)

12.33293399991453
3.0004189998180664
3.8598420001108025
3.1215310000334284
3.3264399999097805
3.982839999935095
3.03716100006568
3.182582999670558
3.4885220002252026
4.193728999780433
3.898957999808772
3.0531930001416185
3.8240390003920766
3.404548000162322
3.811024000242469


3.1949409999469935
3.901550999671599
3.937137999855622
3.5471820001475862
3.4072540001943707
3.3503879999443598
3.3463300001130847
3.149764999761828
3.082779000123992
3.0865569997331477
3.0730830003449228
3.104955999788217
3.066810000291298
3.6008329998367117
3.4215810001114733
2.98112200016476
3.3099809998020646
3.095769000083237
2.5945750003302237
2.825544000188529
2.736748000188527
2.6696889999584528
2.824808000241319
2.623387999847182
2.566345000104775
2.9501150002033683
2.6882100000875653
2.8121639998062165
2.5616349998927035
3.3435540003665665
3.0616740000368736
2.504047000002174
2.726427000197873
2.9552930000136257
2.921284999956697
2.721369000028062
2.552916000240657
2.5087049998546718
3.011182000136614
3.2308359996022773
3.1106599999475293
3.6521249999168504
2.9893329997321416
3.3100529999501305
3.6178139998810366
3.232023999771627
3.063506999751553
3.199064999989787
3.1988639998417057
3.3336640003653883
3.0549739999514713
2.9856239998480305
2.9925490002824517
2.97814799978368